In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Ibrahim Alassaf\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
# Import the neccessery libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Ibrahim
[nltk_data]     Alassaf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Ibrahim
[nltk_data]     Alassaf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
Title = "20_newsgroups"

In [25]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import os 
import string 
import copy 
import pickle 

title = "alt.atheism"
os.chdir("C:/Users/Ibrahim Alassaf/Desktop/lab6/informationRetrieval_lab6/20_newsgroups")
paths = [] 
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'): 
 for i in filenames:  
    paths.append(str(dirpath)+str("\\")+i)

print(dirpath)
paths[0]

C:\Users\Ibrahim Alassaf\Desktop\lab6\informationRetrieval_lab6\20_newsgroups/alt.atheism/


'C:\\Users\\Ibrahim Alassaf\\Desktop\\lab6\\informationRetrieval_lab6\\20_newsgroups/alt.atheism/\\49960'

# Removing stop words

In [26]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ''
    for w in words:
        if w not in stop_words:
            new_text = new_text + ' ' + w
    return np.char.strip(new_text)

# Removing punctuation

In [27]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

# Convert to lowercase

In [28]:
def convert_lower_case(data):
    return np.char.lower(data)

# Stemming 

In [29]:
def stemming(data):
    stemmer = PorterStemmer()

    tokens = word_tokenize(str(data))
    new_text = ''
    for w in tokens:
        new_text = new_text + ' ' + stemmer.stem(w)
    return np.char.strip(new_text)

# Converting numbers to its equivalent words 

In [30]:
def convert_numbers(data):
    data = np.char.replace(data, '0', ' zero ')
    data = np.char.replace(data, '1', ' one ')
    data = np.char.replace(data, '2', ' two ')
    data = np.char.replace(data, '3', ' three ')
    data = np.char.replace(data, '4', ' four ')
    data = np.char.replace(data, '5', ' five ')
    data = np.char.replace(data, '6', ' six ')
    data = np.char.replace(data, '7', ' seven ')
    data = np.char.replace(data, '8', ' eight ')
    data = np.char.replace(data, '9', ' nine ')
    return data


# Removing header 

In [31]:
def remove_header(data): 
    try: 
        ind = data.index('\n\n') 
        data = data[ind:] 
    except: 
        print("No Header") 
    return data

# Removing apostrophe  

In [32]:
def remove_apostrophe(data): 
    return np.char.replace(data, "'", "")

# Removing single characters 

In [33]:
def remove_single_characters(data): 
    words = word_tokenize(str(data)) 
    new_text = "" 
    for w in words: 
        if len(w) > 1: 
            new_text = new_text + " " + w 
    return np.char.strip(new_text)

# Exercise 1: Create a Function preprocess () and invoke the functions above

In [34]:
def preprocess(data, query):
    if not query:
        data = remove_header(data) # remove header 
        data = convert_lower_case(data) # lower case function 
        data = convert_numbers(data) # convert numbers 
        data = remove_punctuation(data) # remove punctuation 
        data = remove_stop_words(data) # remove stop words 
        data = remove_apostrophe(data) # remove apostrophe 
        data = remove_single_characters(data) # remove single characters 
        data = stemming(data) # stemming  
    return data

# Generating Postings for Unigram inverted index 

<ol>
<li>Use a document ID for all the documents (usually the index during iteration)</li>
<li>Preprocess the text</li>
<li>Generate tokens</li>
<li>For every token in the document, add a documentID.</li>
<li>Repeat this for every document.</li>
<li>If the encountered token is a new token which is not in the dictionary, add the token to the dictionary.</li>
<li>The document IDs should be sorted for easy extraction (but depends on the implementation)</li>
<li>Also keep a track of the number of documents that each word consists (will be used for smart Boolean query extraction) </li>
<li>Handle the wrong/nonexistent tokens </li>
</ol>

In [35]:
doc = 0
postings = pd.DataFrame()

for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)

    if doc % 100 == 0:
        print(doc)


    tokens = word_tokenize(str(preprocessed_text))
    for token in tokens:
        if token in postings:
            p = postings[token][0]
            p.add(doc)
            postings[token][0] = p
        else:
            postings.insert(value=[{doc}], loc=0,column=token)
    doc += 1


0
100
200
300
400
500
600
700
800
No Header
900


In [36]:
postings

,kneel,hinduism,revert,infur,amok,trembl,portray,ingersol,mencken,voltair,...,nine,decemb,one,modifi,last,alt,resourc,atheism,name,archiv
0,{999},{999},{999},{999},{999},{999},{999},{999},{999},{999},...,"{0, 1, 2, 4, 5, 17, 18, 19, 22, 24, 27, 29, 30...","{0, 656, 640}","{0, 1, 2, 3, 4, 5, 7, 8, 13, 14, 16, 17, 18, 1...","{0, 1, 414, 867, 900, 901, 713, 490, 203, 204,...","{0, 1, 772, 900, 901, 263, 264, 391, 647, 145,...","{0, 1, 129, 130, 900, 901, 134, 135, 137, 139,...","{0, 98, 900, 901, 203, 205, 847, 976, 821, 662...","{0, 1, 5, 6, 517, 532, 538, 29, 542, 36, 37, 4...","{0, 1, 512, 7, 524, 34, 36, 555, 559, 561, 50,...","{0, 1, 900, 901, 203, 204, 205, 670}"


# Save the output:

In [37]:
postings.to_pickle(title + "_unigram_postings") 

# Read first the stored posting list:

In [38]:
postings = pd.read_pickle(title + "_unigram_postings") 

# Exercise 2: explore posting list. Find how many times the 
following words appear in the posting list [‘one’, ‘nine’, 
‘exam’] 

In [40]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",len(postings[preprocessed_word][0]))
    print("Postings List:",(postings[preprocessed_word][0]))

get_word_postings("one")


one
Frequency: 844
Postings List: {0, 1, 2, 3, 4, 5, 7, 8, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 34, 35, 36, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 114, 119, 121, 122, 126, 128, 129, 130, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 170, 171, 172, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 195, 196, 197, 198, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 222, 223, 224, 225, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 239, 240, 241, 242, 243, 244, 245, 247, 248, 249, 250, 251, 252, 253, 254, 255, 25

# Exercise 2: What is the posting list of “NOT exam”?

In [44]:
def get_not(word):
    a = postings[word][0]
    b = set(range(len(paths)))
    return b.difference(a)

get_not("nine")

{3,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 20,
 21,
 23,
 25,
 26,
 28,
 32,
 33,
 35,
 38,
 39,
 41,
 43,
 45,
 49,
 50,
 51,
 52,
 54,
 55,
 60,
 61,
 69,
 71,
 76,
 77,
 88,
 89,
 97,
 98,
 101,
 104,
 106,
 107,
 109,
 110,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 123,
 124,
 125,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 136,
 138,
 144,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 155,
 156,
 157,
 159,
 165,
 166,
 171,
 172,
 173,
 176,
 182,
 184,
 185,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 197,
 198,
 199,
 207,
 209,
 210,
 211,
 212,
 214,
 216,
 219,
 220,
 221,
 223,
 226,
 227,
 231,
 232,
 235,
 236,
 238,
 243,
 245,
 246,
 248,
 249,
 251,
 252,
 253,
 254,
 256,
 257,
 258,
 260,
 261,
 266,
 267,
 272,
 274,
 278,
 279,
 280,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 300,
 303,
 304,
 305,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 318,
 323,
 324,
 325,
 326,
 330,
 331,
 332,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 34

# Exercise 4. Write code that is able to retrieve a complex 
query such as “exam and not resourc.?

In [45]:
def generate_command_tokens(query):
    query = query.lower()
    tokens = word_tokenize(query)

    comands = []
    query_words = []

    for t in tokens:
        if t not in ['and','or','not']:
            processed_word = preprocess([t],True)
            print(str(processed_word))
            query_words.append(str(processed_word))
        else:
            commands.append(t)

    return commands, query_words

In [48]:
def gen_not_tuple(query_words, commands):

    tup = []

    while 'not' in commands:

        i = commands.index('not')

        word = query_words[i]

        word_postings = get_not(word)

        tup.append(word_postings)

        commands.pop(i)

        query_words[i] = i

        print('\nAfter Not Processing:', commands, query_words)

    return tup


def binary_operations(query_words, commands, tup):

    a = postings[query_words[0]][0]

    query_words.pop(0)

    for i in range(len(commands)):

        if type(query_words[i]) == int:

            b = tup.pop(0)
        else:

            b = postings[query_words[i]][0]

        if commands[i] == 'and':

            a = a.intersection(b)
        elif commands[i] == 'or':

            a = a.union(b)
        else:

            print('Invalid Command')

    return a


def execute_query(query):

    (commands, query_words) = generate_command_tokens(query)

    tup = gen_not_tuple(query_words, commands)

    print('\nCommands:', commands)

    print('\nQuery Words:', query_words)

    print('\nTup:', len(tup))

    final_set = binary_operations(query_words, commands, tup)

    print('\nFinal Set:', final_set)

    return final_set


def print_file(file):

    out_file = open(paths[file], 'r', encoding='cp1250')

    out_text = out_file.read()

    print(out_text)


In [50]:
query = "exam and not resourc"

In [ ]:
lists = execute_query(query)